# Day 16

In [1]:
with open('../inputs/adventofcode.com_2022_day_16_input.txt', 'r') as f:
    data = f.read().splitlines()

print(f'The file contains {len(data)} lines.')

The file contains 56 lines.


In [2]:
# Imports
import re
from collections import deque

## Puzzle 1

In [3]:
# Valve class
class Valve():
    def __init__(self, name, rate, tunnels):
        self.name = name
        self.rate = rate
        self.tunnels = tunnels

    def __repr__(self):
        return f'Valve {self.name} has flow rate={self.rate}; tunnels lead to valves {", ".join(self.tunnels)}'


In [4]:
pattern = re.compile(r'^Valve (?P<name>[A-Z]{2}) has flow rate=(?P<rate>\d+); (?:tunnels lead to valves|tunnel leads to valve) (?P<valves>[A-Z, ]*)$')

# For each line in data, exract the information and store it in a dictionary
valves = {}
for line in data:
    match = pattern.match(line)
    # Extract the valve name
    valve_name = match["name"]
    # Extract the flow rate
    rate = int(match["rate"])    
    # Extract the tunnels
    tunnels = match["valves"].split(", ")
    # Store the valve information in the dictionary
    valves[valve_name] = Valve(valve_name, rate, tunnels)

print(f'There are {len(valves)} valves.')


There are 56 valves.


In [5]:
# Function to comput the distance between two valves
def compute_distance(valve1, valve2, visited=[]):
    # If the valve is a directly connected
    if valve2.name in valve1.tunnels:
        return 1
    
    else:
        visited.append(valve1)
        # Check for each of the tunnels of v1
        distances = []
        for t in valve1.tunnels:
            if valves[t] not in visited:
                distances.append(compute_distance(valves[t], valve2, visited[:]))
                
        # If there's no direct connection, return max distance
        if len(distances) == 0:
            return len(valves)
        else:
            return min(distances) + 1
    

# Create a dictionary to store the distances
dist = {v: {valves[t]: 1 for t in v.tunnels} for v in valves.values()}

for valve in valves.values():
    for v2 in valves.values():
        if valve != v2 and v2 not in dist[valve].keys() and v2.rate > 0:
            d = compute_distance(valves[valve.name], valves[v2.name], [])
            dist[valve][v2] = d


In [6]:
class step:
    def __init__(self, valve, minutes_left, pressure, visited):
        self.valve = valve 
        self.minutes_left = minutes_left
        self.pressure  = pressure
        self.visited = visited 
        
    def explore(self):
        for valve, steps in dist[self.valve].items():
            time_left = self.minutes_left
            if valve.rate > 0 and valve not in self.visited and (time_left := self.minutes_left - steps - 1) > 0:
                yield __class__(valve, 
                           time_left, 
                           self.pressure + valve.rate * time_left, 
                           frozenset(self.visited) | frozenset({valve}),
                           )
                

max_relief = {}

# We use breadth-first search to explore the possible paths
queue = deque([step(valves['AA'], 30, 0, frozenset({valves['AA']}))])

while queue:
    node = queue.popleft()
    for new_node in node.explore():
        max_relief[new_node.visited] = max(
                    max_relief.get(new_node.visited, 0), new_node.pressure
                )
        queue.append(new_node)
        
print(max(max_relief.values()))

1460


## Puzzle 2

In [7]:
class step:
    def __init__(self, valve, minutes_left, pressure, visited):
        self.valve = valve 
        self.minutes_left = minutes_left
        self.pressure  = pressure
        self.visited = visited 
        
    def explore(self):
        for valve, steps in dist[self.valve].items():
            time_left = self.minutes_left
            if valve.rate > 0 and valve not in self.visited and (time_left := self.minutes_left - steps - 1) > 0:
                yield __class__(valve, 
                           time_left, 
                           self.pressure + valve.rate * time_left, 
                           frozenset(self.visited) | frozenset({valve}),
                           )
                
max_relief = {}

queue = deque([step(valves['AA'], 26, 0, frozenset({}))])

while queue:
    node = queue.popleft()
    # print(valves_left)
    for new_node in node.explore():
        queue2 = deque([step(valves['AA'], 26, 0, frozenset(new_node.visited))])
        while queue2:
            node2 = queue2.popleft()
            for new_node2 in node2.explore():
                visited_nodes = new_node.visited | new_node2.visited

                max_relief[visited_nodes] = max(
                            max_relief.get(visited_nodes, 0), new_node.pressure + new_node2.pressure
                        )
                queue2.append(new_node2)
        queue.append(new_node)
        
print(max(max_relief.values()))

2117
